# Plot fANOVA from the csv file of the HPO results
If you want to run this notebook, I recommend doing it in a separate virtual environment to avoid library dependency issues.

In [ ]:
import random

import ConfigSpace
from ConfigSpace.hyperparameters import UniformFloatHyperparameter
from fanova import fANOVA
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
np.float = float

In [ ]:
pd.set_option('display.max_columns', 50)
sns.set_theme(style="darkgrid")
sns.set_context("notebook", font_scale=1.5, rc={"font.size":20,"axes.titlesize":20,"axes.labelsize":20}) 
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 20
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['legend.frameon'] = True

In [ ]:
seed = 100
random.seed(seed)
np.random.seed(seed)

In [ ]:
df = pd.read_csv("final_score.csv").drop(columns=["Unnamed: 0"])
df = df[df["algorithm"] == "adaptive-trpo"]
df

In [ ]:
env = "walker2d"  # "hopper"

In [ ]:
df_env = df[df["env"] == env].reset_index(drop=True)

In [ ]:
x = df_env[["init_target_kl", "target_snr", "update_strength", "smoothing_coeff"]]
y = df_env["rewards"]

In [ ]:
x

In [ ]:
y

In [ ]:
config_space = ConfigSpace.ConfigurationSpace()
config_space.add(UniformFloatHyperparameter(name="init_target_kl", log=True, lower=0.0001, upper=0.1))
config_space.add(UniformFloatHyperparameter(name="target_snr", log=False, lower=1.0, upper=5.0))
config_space.add(UniformFloatHyperparameter(name="update_strength", log=False, lower=0.0001, upper=0.1))
config_space.add(UniformFloatHyperparameter(name="smoothing_coeff", log=False, lower=0.0, upper=1.0))

In [ ]:
f = fANOVA(x, y, config_space=config_space, seed=seed)

In [ ]:
f.quantify_importance(dims=("init_target_kl", "target_snr", "update_strength", "smoothing_coeff"))

In [ ]:
df_fractions = pd.DataFrame()
for i in range(4):
    dim = (i,)
    fractions_individual = np.array([f.V_U_individual[dim][t] / f.trees_total_variance[t] for t in range(f.n_trees)])
    df_fractions[f.cs.at[i]] = fractions_individual

In [ ]:
df_fractions

In [ ]:
xticks_dict = {"init_target_kl": r"$\delta_{\text{init}}$", "smoothing_coeff": r"$\beta$", "target_snr": r"$\alpha$", "update_strength": r"$\lambda$"}

In [ ]:
fig = sns.boxplot(df_fractions)

xticks = fig.get_xticklabels()
fig.set_xticklabels([xticks_dict[s.get_text()] for s in xticks])

fig.figure.savefig(f"fanova_{env}.pdf", bbox_inches="tight")